## Imports

In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

## Dataset Loading

In [7]:
# Paths
train_dir = '../dataset/plant_dataset/train'
valid_dir = '../dataset/plant_dataset/valid'

# Image Data Generator with real-time augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

valid_datagen = ImageDataGenerator(rescale=1./255)

# Create generators
train_gen = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')
valid_gen = valid_datagen.flow_from_directory(valid_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')

Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.


## Model Building (Transfer Learning with MobileNetV2)

In [8]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

# Add custom top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
preds = Dense(train_gen.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=preds)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


## Model Training

In [9]:
# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.2)
checkpoint = ModelCheckpoint("outputs/saved_model/best_model.h5", save_best_only=True)

# Training
history = model.fit(train_gen,
                    epochs=20,
                    validation_data=valid_gen,
                    callbacks=[early_stop, lr_reduce, checkpoint])

C:\Users\etern\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
  18/2197 ━━━━━━━━━━━━━━━━━━━━ 27:55 769ms/step - accuracy: 0.0571 - loss: 3.7765

KeyboardInterrupt: 

## Evaluation Metrics

In [ ]:
# Predict on validation set
y_true = valid_gen.classes
y_pred = model.predict(valid_gen)
y_pred_labels = np.argmax(y_pred, axis=1)

# Classification report
print(classification_report(y_true, y_pred_labels, target_names=list(valid_gen.class_indices.keys())))

# F1-score
f1 = f1_score(y_true, y_pred_labels, average='macro')
print("Macro F1-score:", f1)

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred_labels)

# Plot confusion matrix
plt.figure(figsize=(20,20))
sns.heatmap(cm, annot=False, cmap="Blues", xticklabels=valid_gen.class_indices.keys(),
            yticklabels=valid_gen.class_indices.keys())
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

## Training History Visualization

In [ ]:
# Accuracy
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.title('Accuracy over Epochs')
plt.show()

# Loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss over Epochs')
plt.show()

## Save Model

In [ ]:
model.save('outputs/saved_model/plant_disease_model_final.h5')